### Preprocessing

scRNA-Seq of Placental explants

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib
import os
import sys
%matplotlib inline
import seaborn as sns
import seaborn as sb
from glob import iglob
import anndata
import matplotlib as mpl
import skmisc

    
sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.figdir = '../results/'
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

sys.executable


fig_path= '../results/'

-----
anndata     0.7.5
scanpy      1.7.1
sinfo       0.3.1
-----
PIL                 8.1.2
anndata             0.7.5
anyio               NA
attr                20.3.0
babel               2.9.0
backcall            0.2.0
brotli              NA
cairo               1.20.0
certifi             2020.12.05
cffi                1.14.5
chardet             4.0.0
cloudpickle         1.6.0
colorama            0.4.4
cycler              0.10.0
cython_runtime      NA
cytoolz             0.11.0
dask                2021.03.1
dateutil            2.8.1
decorator           4.4.2
fsspec              0.8.7
get_version         2.1
google              NA
h5py                3.1.0
idna                2.10
igraph              0.8.3
ipykernel           5.5.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              2.11.3
joblib              1.0.1
json5               NA
jsonschema          3.2.0
jupyter_server      1.4.1
jupyterlab_server   2.3.0
kiwisolver          1.3.1


In [2]:
def Barplot(which_var, adata, var='identity', height=3, color = False, suffix= ''):
    
    '''
    Function to plot barplots plotting the proportion of cells per catergory in var, coming from each category in which_var.
    
    Parameters:
        which_var: column name in .obs. Contains the categories to contrast.
        adata: anndata object.
        var: column name in .obs. It contains the categories of the cells.
        height: plot height
        color: colors to use
        suffix: string. Suffix to be added at the end of the name of the plot.
    
    Return:
        Saves bar plot as a pdf.
        
    '''
    
    plotdata = pd.crosstab(adata.obs[var], adata.obs[which_var], normalize='index') * 100
    if 'category' in plotdata.index.dtype.name:
        plotdata.index.reorder_categories(adata.obs[var].cat.categories[::-1])

    if not color:
        ax1 = plotdata.plot.barh(stacked = True, edgecolor = 'none', zorder = 3, figsize = (6,height), fontsize = 14, grid = False)
    else:
        ax1 = plotdata.plot.barh(stacked = True, edgecolor = 'none', zorder = 3, figsize = (6,height), fontsize = 14, grid = False, color = color)
    ax1.set_title(which_var+' %')
    ax1.set_ylabel(var)
    horiz_offset = 1
    vert_offset = 1.
    ax1 = ax1.legend(bbox_to_anchor = (horiz_offset, vert_offset))
    ax1.figure.savefig(str(sc.settings.figdir)+'/barplot_'+var+'_proportions_'+which_var+ suffix+'.pdf', bbox_inches='tight',
                       dpi=300, orientation='landscape', format= 'pdf', optimize=True)

In [3]:
### Function to add the souporcell cluster
####

def add_souporcell_id(adata_obj, cell_id, samples):
    '''
    Function to add the souporcell status (e.g. 0,1, not pooled, etc.) to the cells
    
    Input
        adata_obj: adata object
        cell_id: id of a cell
        samples: list of samples that were pooled. Usually not all samples are multiplexed.
    
    '''
    
    curr_sample = adata_obj.obs.loc[cell_id, 'sample']
    
    #print('sample',sample, 'barcode', cell_id)
    
    if curr_sample in samples:
        #extracts the table contained in the indicated key of the dictionary
        curr_souporcell_table = souporcell_clusters[curr_sample]
        
        if (cell_id in list(curr_souporcell_table.index)): #checking that the cells are into the data
            curr_assign = souporcell_clusters[curr_sample].loc[cell_id,'assignment']
            #print('returning',curr_assign)
            return(curr_assign)

        else:
            # cell barcode is filtered by souporcell
            return('filtered_by_souporcell')
    else:
        return('not_pooled')

In [4]:
def decode_donors(adata_obj, cell_id, identity_dict,samples):

    '''
    Function to add the true identity to the multiplexed samples. The souporcell samples are changed
    to the real names of the clusters(e.g. 0= Hrv99, 1=Hrv98).
    
    Input
        adata_obj: adata object
        cell_id: id of a cell
        identity_dict: dictionary with the identity of each cluster per each sample. It has the structure sample:{cluster:donor}.
        samples: list of samples to be considered in the function. Not all samples are multiplexed
    '''
    
    
    #sample name
    curr_sample = adata_obj.obs.loc[cell_id, 'sample']
    
    #cluster name
    curr_souporcell_cluster = adata_obj.obs.loc[cell_id, 'assignment_SoC']

    if curr_sample in samples:
        #this means that the cell could not be assigned to a cluster (e.g. 2/3), therefore it is considered as a doublet
        if '/' in curr_souporcell_cluster:
            return('donor_doublets')
        
        #Condition for cells that were filtered by SoC. These will be deleted later.
        elif "filtered" in curr_souporcell_cluster:
            return('filtered_by_souporcell')
        
        #Singlets with a donor assigned
        else:
            return(identity_dict[curr_sample][curr_souporcell_cluster])


        


### Import data

In [5]:
data_dir ='../data/'
metadata_dir ='../metadata/'

meta = pd.read_csv(metadata_dir+'meta_exp_infection_Tg_scell.csv',index_col=0)
meta['donor'] = meta['donor'].astype('str')
plotmeta = list(meta.columns)
plotmeta.append('sample')
print('Number of samples: ', meta.index.size)

Number of samples:  4


In [6]:
meta

,stage,donor,hpi,infection
sample,,,,
Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI
Pla_HDBR13007975,Tg_24h,Hrv171_Hrv172,24h,Tg
Pla_HDBR13798223,UI_24h,Hrv241_Hrv242,24h,UI
Pla_HDBR13798224,Tg_24h,Hrv241_Hrv242,24h,Tg


### Preprocessing

No basic filtering at this stage to keep integrate the parasite genes

Quantify: 1) % mitochondrial genes; 2) total counts

In [7]:
holder = []
for sample in meta.index:
    print(sample)
    # Load 10x data as AnnData

    holder.append(sc.read_10x_mtx(data_dir+sample,var_names='gene_symbols',cache=True)) 
    print('Original number of cells: {:d}'.format(holder[-1].n_obs))
    
    holder[-1].var_names_make_unique()
    holder[-1].obs_names = [sample+'_'+i.split('-')[0] for i in holder[-1].obs_names]

    
    # add in metadata
    holder[-1].obs['sample'] = sample
    for val in meta.columns:
        holder[-1].obs[val] = meta[val][sample]
    
    # Extract mitochondial genes
    mito_genes = [name for name in holder[-1].var_names if name.startswith('MT-')]
    #for each cell compute fraction of counts in mito genes vs. all genes
    #the `.A1` is only necessary, as X is sparse - it transform to a dense array after summing
    holder[-1].obs['percent_mito'] = np.sum(
         holder[-1][:, mito_genes].X, axis=1).A1 / np.sum(holder[-1].X, axis=1).A1
    
    #add the total counts per cell as observations-annotation to adata
    holder[-1].obs['n_counts'] = holder[-1].X.sum(axis=1).A1
    
    print('Total number of cells: {:d}'.format(holder[-1].n_obs))
    print('Total number of genes: {:d}'.format(holder[-1].n_vars))

Pla_HDBR13007974
Original number of cells: 16274
Total number of cells: 16274
Total number of genes: 45238
Pla_HDBR13007975
Original number of cells: 17962
Total number of cells: 17962
Total number of genes: 45238
Pla_HDBR13798223
Original number of cells: 16936
Total number of cells: 16936
Total number of genes: 45238
Pla_HDBR13798224
Original number of cells: 16642
Total number of cells: 16642
Total number of genes: 45238


In [8]:
# confirm N samples
print(len(holder))
# merge datasets by taking the intersection of cells found between cells 
adata = holder[0].concatenate(holder[1:],join='outer',index_unique=None)
# copy of this matrix in Compressed Sparse Row format
adata.X = adata.X.tocsr()
adata

4


AnnData object with n_obs × n_vars = 67814 × 45238
    obs: 'sample', 'stage', 'donor', 'hpi', 'infection', 'percent_mito', 'n_counts', 'batch'
    var: 'gene_ids', 'feature_types'

In [9]:
adata.obs.head()

,sample,stage,donor,hpi,infection,percent_mito,n_counts,batch
Pla_HDBR13007974_AAACCCAAGAAGAGCA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.001482,5397.0,0
Pla_HDBR13007974_AAACCCAAGCGTTGTT,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.019590,6432.0,0
Pla_HDBR13007974_AAACCCAAGTAGTCAA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.045489,49221.0,0
Pla_HDBR13007974_AAACCCACAATGAACA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.045332,9243.0,0
Pla_HDBR13007974_AAACCCACAGAGAGGG,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.031214,7753.0,0


# Scrublet

In [10]:
scorenames = ['scrublet_score','scrublet_cluster_score','zscore','bh_pval','bonf_pval']

scrdf = []
for sample in meta.index:
    #reading the scrublet scores done in S0
    scrdf.append(pd.read_csv('../results/scrublet-scores/'+sample+'.csv', header=0, index_col=0))

scrdf = pd.concat(scrdf)
scrdf.index = [i.replace('-1', '') for i in scrdf.index]
for score in scorenames:
    adata.obs[score] = scrdf[score]

# In scrublet the significant p-value mark doublets
adata.obs['is_doublet'] = adata.obs['bonf_pval'] < 0.01

#### Q1: What is the number of cells detected as doublets?

In [11]:
print('Total number of doublets: {:d}'.format(len(adata.obs.loc[adata.obs['is_doublet'] == True])))

Total number of doublets: 2201


In [12]:
adata.obs.head()

,sample,stage,donor,hpi,infection,percent_mito,n_counts,batch,scrublet_score,scrublet_cluster_score,zscore,bh_pval,bonf_pval,is_doublet
Pla_HDBR13007974_AAACCCAAGAAGAGCA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.001482,5397.0,0,0.022316,0.027244,-1.040029,0.956054,1.0,False
Pla_HDBR13007974_AAACCCAAGCGTTGTT,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.019590,6432.0,0,0.020057,0.029250,-0.970364,0.946081,1.0,False
Pla_HDBR13007974_AAACCCAAGTAGTCAA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.045489,49221.0,0,0.092913,0.083395,0.909816,0.899034,1.0,False
Pla_HDBR13007974_AAACCCACAATGAACA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.045332,9243.0,0,0.085202,0.073439,0.564082,0.899034,1.0,False
Pla_HDBR13007974_AAACCCACAGAGAGGG,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.031214,7753.0,0,0.099262,0.070423,0.459349,0.909845,1.0,False


#### Concatenating columns and creating new column showing Infections + stage

In [13]:
adata.obs["infection_stage"] = adata.obs["infection"].astype('string') +'_'+ adata.obs["hpi"]

In [14]:
adata.obs["infection_stage"] = adata.obs["infection_stage"].astype("category")

In [15]:
set(adata.obs["infection_stage"])

{'Tg_24h', 'UI_24h'}

In [16]:
# adding the raw counts to a layer
adata.layers['raw_counts']=adata.X

## Calculate cell-cycle scores

We here perform cell cycle scoring. To score a gene list, the algorithm calculates the difference of mean expression of the given list and the mean expression of reference genes. To build the reference, the function randomly chooses a bunch of genes matching the distribution of the expression of the given list. Cell cycle scoring adds three slots in data, a score for S phase, a score for G2M phase and the predicted cell cycle phase.

First read the file with cell cycle genes, from Regev lab and split into S and G2M phase genes. Cell cycle genes were retrieved from the scanpy_usage github site via web browser at RegevLab Github repo.

In [17]:
#Normalizing the data and scaling for the cell-cycle scoring
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
sc.pp.log1p(adata)


# Scale
sc.pp.scale(adata, max_value=10)

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [18]:
# calculate cell cycle scores

cell_cycle_genes = [x.strip() for x in open('../metadata/regev_lab_cell_cycle_genes.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
s_genes = [x for x in s_genes if x in adata.var_names]
g2m_genes = [x for x in g2m_genes if x in adata.var_names]

sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/conda/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


In [19]:
adata.obs

,sample,stage,donor,hpi,infection,percent_mito,n_counts,batch,scrublet_score,scrublet_cluster_score,zscore,bh_pval,bonf_pval,is_doublet,infection_stage,S_score,G2M_score,phase
Pla_HDBR13007974_AAACCCAAGAAGAGCA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.001482,5397.0,0,0.022316,0.027244,-1.040029,0.956054,1.0,False,UI_24h,-0.168818,-0.049019,G1
Pla_HDBR13007974_AAACCCAAGCGTTGTT,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.019590,6432.0,0,0.020057,0.029250,-0.970364,0.946081,1.0,False,UI_24h,-0.179548,-0.216433,G1
Pla_HDBR13007974_AAACCCAAGTAGTCAA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.045489,49221.0,0,0.092913,0.083395,0.909816,0.899034,1.0,False,UI_24h,-0.016134,-0.171390,G1
Pla_HDBR13007974_AAACCCACAATGAACA,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.045332,9243.0,0,0.085202,0.073439,0.564082,0.899034,1.0,False,UI_24h,0.031297,-0.087906,S
Pla_HDBR13007974_AAACCCACAGAGAGGG,Pla_HDBR13007974,UI_24h,Hrv171_Hrv172,24h,UI,0.031214,7753.0,0,0.099262,0.070423,0.459349,0.909845,1.0,False,UI_24h,-0.036825,0.040418,G2M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pla_HDBR13798224_TTTGTTGTCCTGTTAT,Pla_HDBR13798224,Tg_24h,Hrv241_Hrv242,24h,Tg,0.703766,1779.0,3,0.015119,0.019177,-3.346372,0.999652,1.0,False,Tg_24h,-0.133383,-0.089154,G1
Pla_HDBR13798224_TTTGTTGTCGTAGAGG,Pla_HDBR13798224,Tg_24h,Hrv241_Hrv242,24h,Tg,0.209441,11101.0,3,0.014669,0.044088,-1.848236,0.999652,1.0,False,Tg_24h,-0.229195,-0.109915,G1
Pla_HDBR13798224_TTTGTTGTCTCATGGA,Pla_HDBR13798224,Tg_24h,Hrv241_Hrv242,24h,Tg,0.072021,45945.0,3,0.061511,0.075568,0.044945,0.973885,1.0,False,Tg_24h,1.578841,-0.010208,S
Pla_HDBR13798224_TTTGTTGTCTCCACTG,Pla_HDBR13798224,Tg_24h,Hrv241_Hrv242,24h,Tg,0.748331,1947.0,3,0.019939,0.022320,-3.157363,0.999652,1.0,False,Tg_24h,-0.145740,-0.021057,G1


#### Q2: How many cells are cycling cells?

In [22]:
print('Cycling cells: {:d}'.format(len(adata.obs.loc[adata.obs['phase'] != "G1"])))

Cycling cells: 26821


In [24]:
print('Not cycling cells: {:d}'.format(len(adata.obs.loc[adata.obs['phase'] == "G1"])))

Not cycling cells: 40993


## Save raw counts

In [20]:
# adding the raw counts
adata.X=adata.layers['raw_counts'].copy()
#adata.raw = adata.copy
adata.write('../data/rna1_counts_Tg_adata.h5ad')

... storing 'sample' as categorical
... storing 'stage' as categorical
... storing 'donor' as categorical
... storing 'hpi' as categorical
... storing 'infection' as categorical
... storing 'phase' as categorical
... storing 'feature_types' as categorical
